# Seq2seq

自然言語処理に関するタスクは、言語の時系列データを別の時系列データに変換するような問題として定式化することができます。
- 機械通訳のタスクでは、ある言語の文章（入力時系列データ）を別の言語の文章（出力時系列データ）に変換します。
- 要約のタスクでは、長いテキスト（入力時系列データ）を短い要約文（出力時系列データ）に変換します。

このように、自然言語処理の多くのタスクは、時間的に順序付けられたデータを取り扱い、それを新しい形式の順序付けられたデータに変換するプロセスとして捉えることができます。このプロセスは、データの時系列的な性質と言語の構造的な特徴を考慮に入れる必要があります。そのためのモデルは、seq2seqと呼ばれます。

## seq2seqの原理

seq2seq(Sequence-to-Sequence)はEncoder-Decoderモデルとも呼ばれます。その名前が示す通り、そこには2つのモジュールー**Encoder**と**Decoder**ーから構成されています。

- エンコードとは、情報をある規則に基づいて変換することを指しています。具体的には、入力シーケンスを取り込み、それを固定長の内部表現（通常はベクトル）に変換することです。この内部表現には、入力シーケンスの重要な情報がコンパクトにまとめられています。
- デコードは、エンコーダーから受け取った特徴ベクトルを基に、目的の出力シーケンスを生成します。この過程で、デコーダーはエンコーダーが提供した情報を「解釈」し、それに基づいて適切な出力を生成するための処理を行います。

<img src="./Figure/encoder-decoder.png" alt="fishy" class="bg-white" width="400px">

```{note}

機械翻訳の歴史

機械翻訳（Machine Translation：MT）は計算機科学の黎明期から長きにわたって取り組まれてきた伝統的な課題であり，自然言語処理研究の発展を牽引しています。

- ルールベース

    - 1950年代から本格的な研究が始められました。当時は、翻訳先の言語の文法規則にもとづくルールベース機械翻訳が使われていました。膨大なルールを作り込む必要があり、開発にも新語のアップデートにも手間がかかりました。精度は高くなく、主に定型的な文しか翻訳できませんでした。

- 統計的機械翻訳（Statistical Machine Translation、SMT）
    - 1980年代になると、さらに統計的機械翻訳が生み出されました。統計的機械翻訳では、人の代わりにコンピュータがルールを学びます。原文と訳文のペアを大量に読み込み、原文の単語やフレーズが訳文の特定の単語やフレーズに変換される確率を計算することで、特定の言葉や表現がどのように翻訳されるかを学習します。


    - ただし、この手法は、文法構造や言語の深い理解に基づく翻訳を生成することが難しいことがあります。そのため、十分な並列コーパスがない場合や、文法構造が大きく異なる言語ペアに対しては性能が低下することがあります。結果として、翻訳されたテキストには文法的な不正確さや不自然さが生じることがあります。

    - SMTでは原言語の部分単語列を目的言語の句に翻訳し，それを適当な順序に並べ替えるという構成になっています。そのため、句の翻訳モデル，句の並べ替えモデル，出力文の言語モデルを用いて、膨大な翻訳結果候補の中からスコアが最大になるものを探索することになります

$$
argmax_{y} P(y|x) = argmax_{y} \underset{\text{Translation Model  }}{P(x|y)}  \underset{\text{  Language Model}}{P(y)}
$$

- ニューラル翻訳(Neural Machine Translation, NMT)

    - 機械翻訳が大きな進展を迎えたのは2014年のseq2seqモデルの登場でした。seq2seqでは単語列をRNNに入力した後RNNに単語列を出力させる，という極めて単純な仕組みで機械翻訳が実現できるという可能性を示しました。

    - SMTが単語や句の単位で翻訳したものを組み合わせて翻訳文を構成しようとするのに対し，NMT はそのように明示的な要素の組合せを考慮する必要がなく、正しい翻訳結果が出力されるようにニューラルネットワークを学習します。

    - NMTにおいては単語や文はベクトルとして多次元連続空間に埋め込まれて処理され，ある出力単語はそれ以前の処理すべての結果を踏まえて選択されるため，具体的にどの入力単語と対応することを考慮する必要がありません。


```

## RNNによるseq2seq

EncoderとDecoderにはRNNを使うことができます。

- EncoderはRNNを利用して、時系列データを$h$という隠れ状態ベクトルに変換します。つまり、Encoderの出力は、RNNレイヤの最後の隠れ状態です。
    - **情報の集約**：最後の隠れ状態$h$に、入力シーケンスの全ての要素から抽出された情報を集約しています。これにより、タスク（ここは機械通訳）を完成するために必要な情報がエンコードされます。
    - **固定長の表現**：RNNによって生成される$h$は固定長のベクトルです。エンコードするとは、任意の長さの文章を固定長のベクトルに変換することなのです。

$$
h_t = EncoderRNN(e(x_t),h_{t-1})
$$


![](./Figure/encoder.png)

```{margin}
固定長のベクトルは、長いシーケンスの情報を効率的に圧縮することで、入力データの重要な特徴を抽出し、無関係または冗長な情報を無視することができます。
また、固定長の表現を使用することで、モデルは異なる長さの入力データを処理する際にも、一貫した方法で情報を処理・伝達することができます。
```

- DecoderもRNN利用して、Decoderの役割は、単純にvector-to-sequence RNNであり、Encoderから受け取った固定長の隠れ状態$h$を使用して、目的の出力シーケンスを生成することです。

    - **出力シーケンスの生成**: Decoderは、Encoderから受け取った隠れ状態を初期状態として利用し、出力シーケンス（例えば翻訳されたテキスト）を一つずつ生成します。

$$
s_t = DecoderRNN(d(y_t),s_{t-1})
$$
ただし、$s_0=h_t$

![](./Figure/decoder.png)

まとめると、RNNによるseq2seqは二つのRNN ー EncoderのRNNとDecoderのRNN　ー　によって構成されます。この時、RNNレイヤの隠れ状態がEncoderとDecoderの「架け橋」となります。

seq2seqの革新的な点は、入力と出力系列の長さ$n_x$と$n_y$について$n_x=n_y$という制約に対して、seq2seqにおいて$n_x$と$n_y$が違っても構わないという点です。

順伝播では、EncoderからDecoderへンコードされた情報がRNNレイヤの隠れ状態を通じて伝わります。そして、seq2seqの逆伝播では、その「架け橋」を通って、勾配がDecoderからEncoderへ伝わります。

![](./Figure/RNN_seq2seq.png)

## seq2seqの汎用性

seq2seqの構造によって、任意の系列データを扱うことが可能になります。系列にモデル化できるのであれば、文章、音声、ビデオなどを含めて、異なる種類のデータやタスクに適応することができます。

- チャットボット: ユーザーの質問やコメント（入力系列）に対して適切な応答（出力系列）を生成するために使用されます。
- 音声認識: 音声の波形（入力系列）をテキストの形式（出力系列）に変換します。
- 要約システム: 長いテキスト（入力系列）を短い要約文（出力系列）に変換するために使用されます。
- 画像キャプション: 画像（入力系列）を解説するテキスト（出力系列）に変換します。